## Perfilamiento con Kale:

In [1]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy
import scipy.io as sio
import time

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

#mex_c
#from mex.mex_c import general_c as mex_c_g
#from mex.mex_c import simplex_networks_c as mex_c_sn
#from mex.mex_c import problem_definition_c as mex_c_pd
#from mex.mex_c.minimizer_class_c import Minimizer_c
#from mex.mex_c.maximizer_class_c import Maximizer_c
#from mex.mex_c import *

## Características de la instancia

Utilizamos una instancia `m5.2xlarge` que entra en el programa de AWS Educate y que hemos ocupado en prácticas anteriores. Esta instancia tiene las siguientes características:

In [2]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3099.345
BogoMIPS:            5000.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

### Cargamos los datos:

In [2]:
mat = sio.loadmat('../data/AGG.mat')

# Perfilamiento de paquete `mex`

## Sin compilación a C

In [22]:
A1_aprox = mat['A']
b_aprox = mat['b']
c_aprox = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
#A.shape

In [23]:
A_aprox = A1_aprox.toarray()

In [24]:
minim = Minimizer(A_aprox, b_aprox, c_aprox)

In [25]:
minim.add_constraints(lbounds,ubounds)
#minim.matrix.shape

In [26]:
start_time = timeit.default_timer()
minim.solve()
secs_mex = timeit.default_timer() - start_time
print("Todo el proceso tomó",secs_mex,"segundos" )

Todo el proceso tomó 77.25020213400057 segundos


## Con compilación a C

In [40]:
minim_c = Minimizer_c(A_aprox,b_aprox,c_aprox)

In [41]:
minim_c.add_constraints(lbounds,ubounds)
#minim_c.matrix.shape

In [42]:
start_time_c = timeit.default_timer()
minim_c.solve()
secs_mex_c = timeit.default_timer() - start_time_c
print("Todo el proceso tomó",secs_mex_c,"segundos" )

Todo el proceso tomó 78.09592265399988 segundos


# Perfilamiento de paquete `scipy`

In [11]:
A2 = mat['A']
b_scipy = mat['b']
c_scipy = mat['c']
lbounds_scipy = mat['lbounds']
ubounds_scipy = mat['ubounds']
A2.shape

(488, 615)

In [12]:
A_scipy = A2.toarray()

In [13]:
minim_scipy = Minimizer(A_scipy, b_scipy, c_scipy)

In [14]:
minim_scipy.add_constraints(lbounds_scipy,ubounds_scipy)
#minim_scipy.matrix.shape

In [15]:
tableau_obj = minim_scipy.matrix

In [16]:
n_restr = tableau_obj.shape[0]-1
n_vars = A_scipy.shape[1]

In [17]:
c_min_obj = tableau_obj[-1,0:n_vars]
A_min_obj = tableau_obj[0:n_restr, 0:n_vars]
b_min_obj = tableau_obj[0:n_restr, -1]

In [18]:
start_time_scipy = timeit.default_timer()
linprog(c_min_obj, A_ub=A_min_obj, b_ub=b_min_obj).fun
secs_scipy = timeit.default_timer() - start_time_scipy
print("Todo el proceso tomó",secs_scipy,"segundos" )

Todo el proceso tomó 302.67399678400034 segundos


## Diferencia de tiempos entre `mex` y `scipy`:

In [44]:
secs_diff = secs_mex - secs_scipy
print("El paquete mex necesito {:.2f}".format(secs_diff),"segundos menos que el paquete scipy para la ejecución!")

El paquete mex necesito -225.42 segundos menos que el paquete scipy para la ejecución!
